In [ ]:
import pandas as pd
import numpy as np
import csv
import sys
import jieba
import time
import jieba.analyse
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
import heapq
from flask import Flask, jsonify, request
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Get Parameters to DataFrame    
    json_ = request.form.to_dict()
    # Pridicting
    name_test = ['{}'.format(json_['text'])]
    name_test = pd.DataFrame(name_test)
    name_test = name_test.rename(columns={0: 'name'})
    name_test['content_cutted'] = name_test['name'].apply(word_cut)
    global code_Tfidf
    code_Tfidf = Data_prepara()
    content_cutted = pd.Series([dictionary.doc2bow(x) for x in name_test['content_cutted']])
    content_cutted2 = pd.Series(map( lambda x: list(map( lambda y: y[0] ,x)),content_cutted))
    testX = model(content_cutted2)
    test_pred_label = clf.predict_proba(testX)
    google_id_dict = dic()
    max_num_index_list = list(map(test_pred_label[0].tolist().index, heapq.nlargest(5, test_pred_label[0].tolist())))
    d = dict()
    for i in range(0,5):
        d[i+1] = '標籤:'+google_id_dict[str(clf.classes_[max_num_index_list[i]])]+',權重:'+str(test_pred_label[0][max_num_index_list[i]])
    return jsonify(d)
            
def word_cut(mytext):
    return list(jieba.cut(mytext))
def dic():
    with open('google-id-dict2.pickle', 'rb') as handle:
        google_id_dict = pickle.load(handle)
    return google_id_dict
def Data_prepara():
    csv.field_size_limit(int(sys.maxsize/10000000000))
    df2=pd.read_csv(r'D:\Mypython\專題\Join-name5(周周版).csv', sep=',', engine='python',encoding='utf_8_sig')
    df2['content_cutted'] = df2['Join_name'].astype('str').apply(word_cut)
    df_Join_name_count = pd.Series([Counter(x) for x in df2['content_cutted']],index=df2['Join_name'].index)
    global dictionary
    dictionary = Dictionary(list(df_Join_name_count))
    code_tokenGybow = pd.Series([dictionary.doc2bow(x) for x in df_Join_name_count],index=df2['Join_name'].index)
    tfidf = TfidfModel(code_tokenGybow)
    code_tokenGyTfidf = pd.Series(tfidf[code_tokenGybow],index=df2['Join_name'].index)
    code_tokenGyTfidfSort = pd.Series(map( lambda x: sorted(x,key=lambda w: w[1],reverse=True),code_tokenGyTfidf),
                                      index=df2['Join_name'].index)
    code_tokenGyTfidfSort2 = pd.Series(map( lambda x: x[:len(x)//2],code_tokenGyTfidfSort),
                                       index=df2['Join_name'].index)
    code_tokenGyTfidfSort3 = pd.Series(map( lambda x: list(map( lambda y: y[0]  ,x)),
                                       code_tokenGyTfidfSort),index=df2['Join_name'].index)
    code_tokenGyTfidfSort4 = list(code_tokenGyTfidfSort3)
    return code_tokenGyTfidfSort4
def word(x):
    stroke = []
    for i in range(0,len(code_Tfidf)):
        c = 0
        for j in x:
            if j in code_Tfidf[i]: #比對分群關鍵字
                c = c + 1
        stroke.append(c)
    return(stroke)
def model(x):
    AA = []
    AA.append(list(x.apply(word)))
    df_AA = pd.DataFrame(AA[0])
    return df_AA

if __name__ == '__main__':
    app.run(host="127.0.0.1",port=6667)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6667/ (Press CTRL+C to quit)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\andy\AppData\Local\Temp\jieba.cache
Loading model cost 0.935 seconds.
Prefix dict has been built succesfully.
D:\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
127.0.0.1 - - [19/Jun/2019 10:11:22] "POST /predict HTTP/1.1" 200 -


In [1]:
from sklearn.externals import joblib
clf = joblib.load("RandomForest_train_model2.pkl")

D:\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
D:\Anaconda3\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
